# Flatiron Phase 5 Project

## Aaron Galbraith

https://www.linkedin.com/in/aarongalbraith \
https://github.com/aarongalbraith

### Submitted: November 20, 2023

## Contents

- **[Business Understanding](#Business-Understanding)<br>**
- **[Data Understanding](#Data-Understanding)**<br>
- **[Data Preparation](#Data-Preparation)**<br>
- **[Exploration](#Exploration)**<br>
- **[Modeling](#Modeling)**<br>
- **[Evaluation](#Evaluation)**<br>
- **[Recommendations](#Recommendations)<br>**
- **[Further Inquiry](#Further-Inquiry)**<br>

# Business Understanding

In 2022 the US Supreme Court ruled in [Dobbs v. Jackson Women's Health Organization](https://www.supremecourt.gov/opinions/21pdf/19-1392_6j37.pdf) that the United States Constitution would no longer confer a right to abortion and that the legality of abortion timelines and procedures would be at the discretion of the individual states and territories. This brought an end to constitutionally enshrined abortion access after Americans had been guaranteed such rights in all American states and territories for roughly two generations, since the 1973 ruling in [Roe v. Wade](https://tile.loc.gov/storage-services/service/ll/usrep/usrep410/usrep410113/usrep410113.pdf). Both immediately and in the short time since the Dobbs decision, [many states have reduced access to abortion, and 14 states have banned abortion entirely](https://reproductiverights.org/maps/abortion-laws-by-state/). [Advocates for birth control access fear that new government controls could move beyond abortion and attempt to restrict birth control access as well](https://www.npr.org/2022/08/16/1117615628/abortion-birth-control-title-x-supreme-court-family-planning-planned-parenthood).

In the new reproductive environment created by this ruling, Americans who are concerned with family planning need to be better informed of their changing options. They can learn a lot from one another by comparing their experiences with different treatments. This is especially crucial in communities where birth control practices are discouraged and some young people in particular may be less informed about birth control methods, their efficacy, and their side effects. Reproductive health advocacy groups such as Planned Parenthood will likely play an even greater role in this new environment.

In 2018, researchers Surya Kallumadi and Felix Gräßer at UC Irvine created the [UCI ML Drug Review dataset](https://www.kaggle.com/datasets/jessicali9530/kuc-hackathon-winter-2018/) after collecting reviews from [Drugs.com](https://www.drugs.com/) that users had written about various drugs between 2008 and 2017. A substantial portion of these reviews addressed birth control and emergency contraception drugs. Each review was accompanied by a user rating and a tally of how many other users found the review useful ("upvotes").

Our project seeks to use this data in any way that informs and serves those in need of reproductive health care. Our analysis of this data can help inform people about which birth control options are seen by users as the most effective and what side effects are commonly associated with them. We can also apply our rating prediction algorithm to analyze discussions — particularly anonymous ones – in online forums such as [Reddit](https://www.reddit.com/r/birthcontrol/) and [Quora](https://www.quora.com/search?q=birth%20control) without formal ratings systems to detect commenters' sentiment toward and experience with various contraceptive methods. This can help organizations such as Planned Parenthood detect prevalent opinions, preferences and concerns that might not otherwise be available to them through traditional patient feedback mechanisms. Thus informed, Planned Parenthood can respond to their patients' concerns before they even arise.

# Data Understanding

## Import files

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud

import html
import contractions

import re

from IPython.display import display

import seaborn as sns

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from pathlib import Path

SEED = 1979

do_grids = True

## Load and briefly explore data set

In [ ]:
d1 = pd.read_csv('../data/drugsComTrain_raw.tsv', delimiter='\t', encoding='latin-1')
d2 = pd.read_csv('../data/drugsComTest_raw.tsv', delimiter='\t', encoding='latin-1')
df = pd.concat([d1,d2]).reset_index().drop(columns=['Unnamed: 0', 'index'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

There are some missing `condition` labels.

### `drugName` feature

In [ ]:
df.drugName.value_counts()

In [ ]:
df.drugName.value_counts().quantile(.90)

There are 3,671 unique drug names, and 10% of the drug names have more than 120 reviews.

In [ ]:
pd.set_option("display.max_rows", None)
print(df.drugName.value_counts())
pd.set_option("display.max_rows", 60)

A casual overview of the drug names indicates that they all seem valid. Some seem to specify drug combinations and/or dosage amounts.

### `condition` feature

In [ ]:
df.condition.value_counts()

In [ ]:
df.condition.value_counts().quantile(.90)

There are 916 unique conditions, and 10% of the conditions have more than 332 reviews.

In [ ]:
pd.set_option("display.max_rows", None)
print(df.condition.value_counts())
pd.set_option("display.max_rows", 60)

Oddly, the condition labels often (always?) omit initial 'F' and terminal 'r'. We can isolate instances of the former by searching for conditions that start with a lower case letter.

We will eventually trim our records to a number of conditions that Planned Parenthood specializes in treating (and perhaps birth control exclusively), but we will need all the records to help us determine missing condition labels. After we have restored (or discarded) all missing condition labels, we can drop the conditions outside the scope of this review.

### `drugName` × `condition` features

In [ ]:
df.groupby('drugName').condition.nunique().value_counts()[:10]

This means that, for example, 1869 drugs treat 1 condition only, etc.

In [ ]:
df.groupby('condition').drugName.nunique().value_counts()[:10]

This means that 188 conditions are treatable by two drugs, etc.

### `review` feature

In [ ]:
df.review.nunique()

This suggests that just over half of the review values are unique. Almost certainly there will be some duplication issues to deal with.

Let's look at several reviews.

In [ ]:
for i in range(5):
    print(df.review[i], '\n-----')

There appear to be escaped characters (e.g. `&#039;`, indicating an apostrophe) and contractions. We can address this now without affecting our analysis.

We'll reset the review texts to unescape these characters and expand all contractions.

Note: This will replace all instances of `ain't` with `are not`, resulting in some subject-verb agreement issues (e.g. `I are not`). This difference will be negligible in our analysis.

In [ ]:
df.review = df.review.apply(lambda x: contractions.fix(html.unescape(x)))

### `rating` feature

In [ ]:
df.rating.value_counts()

In [ ]:
df.rating.hist(bins=df.rating.nunique());

Most of the conditions lie at the extremes, and more of them appear to be at the positive extreme.

### `date` feature

In order to get a better understanding of the `date` feature, we'll convert it to a `datetime` object.

In [ ]:
df['datetime'] = df.date.apply(lambda x: datetime.strptime(x, '%B %d, %Y'))

In [ ]:
df.datetime.hist(bins=80);

In [ ]:
start_date = df.datetime.min().strftime('%B %d, %Y')
end_date = df.datetime.max().strftime('%B %d, %Y')

print('The reviews span specifically from', start_date+',', 'to', end_date+'.')

The reviews began to surge in early 2015.

### `usefulCount` feature

In [ ]:
usefulCountCumulative = list(df.usefulCount.values)
usefulCountCumulative.sort(reverse=True)

X = range(0,len(df),5000)
Y = []
for x in X:
    Y.append(sum(usefulCountCumulative[:x]))
fig, ax = plt.subplots()
ax.plot(X,Y);

In [ ]:
df.usefulCount.value_counts()[:10]

# Data Preparation

## Missing and erroneous condition labels

In this section we will identify all `condition` labels that are either missing or in need of editing.

In [ ]:
len(df[df.condition.isna()])

In [ ]:
df.condition.fillna('missing', inplace=True)

In [ ]:
len(df[df.condition == 'missing'])

We noticed another `condition` label that was meant to indicate missing and should be accordingly changed.

In [ ]:
df.condition = df.condition.apply(lambda x: 'missing' if 'Not Listed' in x else x)

In [ ]:
len(df[df.condition == 'missing'])

We've identified some actual missing `condition` labels, but we noticed there are more `condition` labels that seem suspicious, particularly ones that start with something other than an upper case character. Let's look at all such `condition` labels.

In [ ]:
set(df[(~df.condition.str[0].isin(list(string.ascii_uppercase))) &
   (df.condition != 'missing')
  ].condition)

These fall into three categories:
1. "X users found this comment helpful" should be regarded as an erroneous label and retagged as "missing.

In [ ]:
df.condition = df.condition.apply(lambda x: 'missing' if 'users found' in x else x)

In [ ]:
len(df[df.condition == 'missing'])

2. Labels that show a clipped copy of the `drugName` label and end with a parenthesis should also be regarded as missing. These erroneous labels merely repeat information already available in the `drugName` feature.

In [ ]:
df.condition = df.condition.apply(lambda x: 'missing' \
                                  if x[0] not in list(string.ascii_uppercase) and \
                                  x[-1] in ['(', ')'] \
                                  else x)

In [ ]:
len(df[df.condition == 'missing'])

3. Other `condition` labels appear to omit the first and/or last several characters. We can infer certain corrections here to restore many of the conditions.

In [ ]:
def condition_restore(condition):
    if condition.split()[-1] in ['Disorde', 'eve', 'Shoulde', 'Cance']:
        condition = condition+'r'
    if condition.split()[0] in ['acial', 'ibrocystic', 'ungal', 'amilial', 'ailure', 'ever', \
                                'emale', 'unctional', 'actor', 'ibromyalgia', 'atigue']:
        condition = 'F'+condition
    if condition.split()[0] in ['llicular', 'llicle', 'lic', 'cal']:
        condition = 'Fo'+condition
    if condition.split()[0] in ['mance']:
        condition = 'Perfor'+condition
    if condition.split()[0] in ['zen']:
        condition = 'Fro'+condition
    if condition.split()[0] in ['mis']:
        condition = 'Dermatitis Herpetifor'+condition
    return condition

df.condition = df.condition.apply(lambda x: condition_restore(x))

Let's look at what we have left.

In [ ]:
set(df[(~df.condition.str[0].isin(list(string.ascii_uppercase))) &
   (df.condition != 'missing')
  ].condition)

"von Willebrand's Disease" appears to be a naturally uncapitalized condition. The others have been impossible to restore and will also be regarded as missing.

In [ ]:
df.condition = df.condition.apply(lambda x: 'missing' \
                                  if x[0] not in list(string.ascii_uppercase) and \
                                  x.split()[0] != 'von' \
                                  else x)

In [ ]:
len(df[df.condition == 'missing'])

We will be able to restore more of these missing condition labels after we do some work with duplicates.

## Duplicates

In [ ]:
df.duplicated().value_counts()

In [ ]:
df[df.duplicated()]

In [ ]:
show_duplicates = True

def show_review(index):
    if show_duplicates:
        display(df[df.review == df.loc[index].review][['drugName', 'condition', 'rating', 'date', 'usefulCount']])
    print('\nReview #'+str(index),'| Rating:',df.loc[index].rating,'| Upvotes:',
          df.loc[index].usefulCount,'\n\n'+df.review.loc[index][1:-1])

In [ ]:
show_review(178703)

For reasons we will explore later, we believe this review was submitted twice by the same person, that each instance of it happened to receive 10 upvotes, and that it should correctly be associated with a grand total of 20 upvotes. Because this is one special instance where the review happened to receive 10 upvotes both times, making it a true duplicate of the data set, we will fix the values here, lest it interfere with operations later on.

In [ ]:
df.at[178703, 'usefulCount'] = 20
df.at[191001, 'usefulCount'] = 20
df.drop([131531, 143768], inplace=True)

## Duplicates due to brand / generic pairs

The main type of duplicate we should look out for is records with duplicate reviews, as those likely indicate some kind of actual erroneous duplication. Let's see how many of those there are.

In [ ]:
df.duplicated(subset=['review']).value_counts()

That's a lot!

Let's explore some facets of these duplicates.

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['drugName']))])

The vast majority of duplicate reviews are accounted for by different drug names. Let's explore some examples.

In [ ]:
for ind in df[df.duplicated(subset=df.columns.difference(['drugName']))].index[:5]:
    show_review(ind)

These five examples make clear that the vast majority of duplicates are due to double-entry; (nearly) every review is entered once with its generic name and once with its brand name.

We can use this phenomenon to restore some of the missing condition labels. If a missing condition label is part of such a unique pair, then we can confidently assign it the condition of its pair-mate.

Let's broaden our search to records that duplicate every feature other than drug name and condition.

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['drugName', 'condition']))])

This is how many records are duplicates of other records in all values EXCEPT (POSSIBLY) drug name and condition. If a record is duplicated in this manner, the second (and third, fourth, etc.) instance will be captured in this bucket of dupes.

If we check only this bucket for dupes, we can see whether there are any triplets, etc.

In [ ]:
df_dupes = df[df.duplicated(subset=df.columns.difference(['drugName', 'condition']))]

In [ ]:
len(df_dupes[df_dupes.duplicated(subset=df_dupes.columns.difference(['drugName', 'condition']))])

There is only one.

In [ ]:
df_dupes[df_dupes.duplicated(subset=df_dupes.columns.difference(['drugName', 'condition']))]

In [ ]:
show_review(140144)

There are 6 records with the same review, date, rating, and condition. (The reviews on October 5, 2012, appear to be just a coincidence of the same review wording for a different drug and condition.) Because they're on the *same day*, it seems likely that these reviews were possibly entered repeatedly by the same person. The two with a useful count of 10 are likely a brand/generic pair.

As for the other 4, it's not clear what is going on. We will (would) later discover that there is also some discrepancy as to which of these is a brand or generic name. Since the review text isn't very descriptive, and the useful count is so low, (and it doesn't pertain to the main conditions treated by Planned Parenthood), let's just drop all 4.

In [ ]:
df.drop([60998, 119972, 133212, 140144], inplace=True)

Now we should be able to create a list of pairs of indices of records that match in all features except possibly drug name and condition. To make this run faster, we'll first create a way to sort them by date.

In [ ]:
%%time
# ⏰ record the time for this cell -- usually 11-12 s

# create stripped down dataframe that does not have drug names or conditions
# we don't need these features for this operation because we're checking for matches on all other features
df_pairs = df.drop(columns=['drugName', 'condition']).copy()

# create a list of indices of records that duplicate everything other than drug name and condition
df_dupes = df_pairs[df_pairs.duplicated()].index.tolist().copy()

# create and populate a dictionary whose keys are dates and whose values are indices
dates_dict = {}
# populate dictionary with keys that are dates belonging to the duplicates
for date_ in list(set(df[df.index.isin(df_dupes)].date.tolist())):
    dates_dict[date_] = []
# populate dictionary with values that are indices that are NOT from the duplicate list but DO share that date
for i in df[~df.index.isin(df_dupes)].index:
    dates_dict[df.loc[i].date].append(i)

Now we can use this dates dictionary to sort and identify the pairs.

In [ ]:
%%time
# ⏰ record the time for this cell -- usually 2–4 mins

# create a list of record pairs where each entry is a list of two indices
pairs = []

# iterate over the indices from the dupes list
for i in df_dupes:
    # set the date to the date from index i
    date_i = df.loc[i].date
    # iterate over OTHER indices who share that date
    for j in dates_dict[date_i]:
        # check for a match
        if df_pairs.loc[i].equals(df_pairs.loc[j]) and df.drugName.loc[i] != df.drugName.loc[j]:
            # remove this index from the dates dictionary so we have fewer to search through in later iterations
            dates_dict[date_i].remove(j)
            # add this pair to the pairs list
            pairs.append([i,j])
            break

Let's take a look at several of the pairs we've collected.

In [ ]:
pairs[:10]

Here we'll create a dictionary that matches the index of one pair member to the other member of the pair.

In [ ]:
pairs_dict = {}

for pair in pairs:
    for i in range(2):
        pairs_dict[pair[i]] = pair[1-i]

## Restore missing `condition` labels

We will restore missing `condition` labels in two ways, in order of certainty:

1. For missing values that possess a pair match, we will assign it the condition of its match.
2. For the remaining missing values, we will assign it the condition that is most commonly associated with its drug name.

In [ ]:
len(df[df.condition == 'missing'])

In [ ]:
%%time
# ⏰ record the time for this cell -- usually 10-15 seconds

# iterate over each record pair
for pair in pairs:
    # iterate over each member of the pair
    for i in range(2):
        # identify a pair member whose condition is missing
        if df.loc[pair[i]].condition == 'missing':
            # assign to the pair member the condition of its pair-mate
            df.at[pair[i], 'condition'] = df.loc[pairs_dict[pair[i]]].condition

In [ ]:
len(df[df.condition == 'missing'])

Because it will be useful later, we'll make a feature that names the indicated drug and, if applicable, the paired drug.

This is not a *final* replacement for the drug name feature, but it will allow us to better recognize the relationship between the generic and brand drug names.

In [ ]:
%%time
# ⏰ record the time for this cell -- usually 15-30 seconds

df['ind'] = df.index

def drugList_fix(index, drugName_):
    drugList = [drugName_]
    if index in pairs_dict:
        drugList.append(df.loc[pairs_dict[index]].drugName)
        # alphabetize each drug pair so that we will not mistakenly duplicate e.g. [A,B] & [B,A]
        drugList.sort()
    return drugList

df['drugList'] = df.apply(lambda x: drugList_fix(x.ind, x.drugName), axis=1)

df.drop(columns='ind', inplace=True)

Now we can create a feature that tells us if a record is associated with a paired drug name or not.

In [ ]:
df['isPaired'] = df.drugList.apply(lambda x: True if len(x) > 1 else False)

In [ ]:
len(df[df.isPaired])

In [ ]:
len(df[~df.isPaired])

Because lists confuse certain operations

In [ ]:
df['drugSetString'] = df.drugList.apply(lambda x: x[0] + ' ' + x[1] if len(x) == 2 else x[0])

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['drugName', 'drugSet', 'drugList']))])

With this new feature in place, we can drop one record from each of the brand/generic pairs. The drug name feature will retain only one member of the pair -- unpredictably either the brand or the generic -- which will make this feature more or less useless for the moment.

Before we drop these records, we'll create a bookmark copy of the dataframe.

In [ ]:
# DO NOT re-run this cell out of sequence
# to use the dataframe as it was at this stage, un-comment, run, and re-comment the cell that follows after it.
df_bookmark_1 = df.copy()

In [ ]:
# df = df_bookmark_1.copy()

In [ ]:
df.drop_duplicates(subset=df.columns.difference(['drugName', 'drugSet', 'drugList']), inplace=True)

In [ ]:
show_duplicates = False

For every remaining record with a missing condition, we will assign it the condition that is most common for the drug indicated by that record. (This will not be biased by duplicates from brand/generic pairs, because we have dropped those duplicates.)

This will be the last use we have for conditions *not* treated by Planned Parenthood.

In [ ]:
drugs_w_missing_condition = list(set(df[df.condition == 'missing'].drugSetString))

In [ ]:
len(drugs_w_missing_condition)

In [ ]:
df.drugSetString.nunique()

This applies to some 20% of the drugs. We'll create a dictionary that reports the most common condition for these drugs.

In [ ]:
%%time
# record the time for this cell -- 10-20 seconds

most_common_condition = {}

for drug in drugs_w_missing_condition:
    condition = df[df.drugSetString == drug].condition.value_counts().idxmax()
    if condition == 'missing' and len(set(df[df.drugSetString == drug].condition)) > 1:
        condition = df[(df.drugSetString == drug) &
                       (df.condition != 'missing')
                      ].condition.value_counts().idxmax()
    proportion = round(df[df.drugSetString == drug].condition.value_counts(normalize=True)[0],2)
    most_common_condition[drug] = [condition, proportion]

In [ ]:
most_common_condition['Sildenafil Viagra']

For example, if a review with an unlisted condition is about Viagra, we will assume the condition is Erectile 
Dysfunction.

In [ ]:
len(df[df.condition == 'missing'])

In [ ]:
df['condition'] = df.apply(lambda x: most_common_condition[x.drugSetString][0] \
                           if x.condition == 'missing' \
                           else x.condition, axis = 1)

In [ ]:
len(df[df.condition == 'missing'])

This is how many records there are that still have no condition label. This means the drugs indicated in these records are *only* indicated in references without an indicated condition. As such, there's not really anything we can do with these records, and we may as well drop them.

In [ ]:
df.drop(df[df.condition == 'missing'].index, inplace=True)

In [ ]:
len(df[df.condition == 'missing'])

## Drop records by condition

At this point, we still have more cleaning to do, but we have identified all the conditions that we can, and we won't have any further need for records with certain condition values, so we'll drop them.

In [ ]:
# DO NOT re-run this cell out of sequence
# to use the dataframe as it was at this stage, un-comment, run, and re-comment the cell that follows after it.
df_bookmark_2 = df.copy()

In [ ]:
# df = df_bookmark_2.copy()

Let's take another look at the complete list of conditions and choose which ones to keep.

In [ ]:
df.condition.nunique()

Since there are so many conditions to consider, let's limit this to just conditions with at least 25 reviews.

In [ ]:
pd.set_option("display.max_rows", None)
display(df['condition'].value_counts().loc[lambda x: x >= 25])
pd.set_option("display.max_rows", 60)

In [ ]:
df.drop(df[~df.condition.isin(['Birth Control', 'Emergency Contraception'])].index, inplace=True)

In [ ]:
df.condition.value_counts()

## Pairing generic and brand names

Now that we have a smaller number of records to deal with, we can sort out generic and brand names.

First we'll create a list of all values from the drug name feature. (Some of these have been dropped from the drug name feature itself when we dropped one record from each brand/generic pair, but all of them were included in the drug list feature.)

We'll create two lists: paired drugs (which we will attempt to sort into brand and generic) and single drugs (each of which we will then try to identify as either brand or generic).

In [ ]:
len(df)

In [ ]:
all_drug_lists = df.drugList.tolist()
all_drug_lists.sort()

In [ ]:
len(all_drug_lists)

In [ ]:
all_drug_names = set()

for list_ in all_drug_lists:
    all_drug_names.add(list_[0])
    if len(list_) > 1:
        all_drug_names.add(list_[1])

all_drug_names = list(all_drug_names)

all_drug_names.sort()

In [ ]:
len(all_drug_names)

In [ ]:
# this will create a full list with duplicates
# we need to do this intermediate before moving to the following step to remove duplicates
paired_drug_lists = df[df.isPaired].drugList.tolist()
paired_drug_lists.sort()

In [ ]:
len(paired_drug_lists)

In [ ]:
paired_drug_names = set()

for pair in paired_drug_lists:
    paired_drug_names.add(pair[0])
    paired_drug_names.add(pair[1])

paired_drug_names = list(paired_drug_names)

paired_drug_names.sort()

In [ ]:
unpaired_drug_names = [drug for drug in all_drug_names if drug not in paired_drug_names]

unpaired_drug_names.sort()

In [ ]:
len(paired_drug_names)

In [ ]:
len(unpaired_drug_names)

Together, these two lists of names constitute all the drug names left to sort into brand and generic categories.

In order to sort the list of paired drugs into brand and generic, we'll establish a dictionary whose keys are all the drug names that appear in a generic/brand pair.

In [ ]:
drug_dict = {}

for drug in paired_drug_names:
    drug_dict[drug] = set()

We'll assign values to those keys according to the pairings. For example, if drug name A is in a generic/brand pair with drug name B, then they will appear on each other's list of values in this dictionary.

In [ ]:
for pair in paired_drug_lists:
    drug_dict[pair[0]].add(pair[1])
    drug_dict[pair[1]].add(pair[0])

Let's find out how many of these drug names are associated with exactly one other drug name.

In [ ]:
len({drug for drug in drug_dict if len(drug_dict[drug]) == 1})

That should mean that exactly the remainder are associated with multiple drug names. It would make sense that drug names that belong to multiple generic/brand pairs are themselves the generic name. On that assumption, we'll create a list of generic drug names.

In [ ]:
generics = [drug for drug in drug_dict if len(drug_dict[drug]) > 1]

generics.sort()

In [ ]:
len(generics)

Now we'll check to make sure that the drug names we've just designated as "generic" do NOT belong to a generic/brand pair with *another* "generic".

In [ ]:
count = 0

for drug in generics:
    for match in drug_dict[drug]:
        if match in generics:
            count += 1

print(count)

Great.

Then we can begin designating drug names as "brands" if they are in a generic/brand pair with a generic.

In [ ]:
brands = set()

for generic in generics:
    for match in drug_dict[generic]:
        brands.add(match)

brands = list(brands)

brands.sort()

In [ ]:
len(brands)

Now let's see what drugs remain and how many records they are associated with.

In [ ]:
uncategorized = list(set(drug for drug in paired_drug_names if drug not in generics and drug not in brands))

uncategorized.sort()

To be clear, these are drug names with the following properties:

- the drug name belongs to an exclusive brand/generic pair
- we have not yet identified which members of the pair are brand and generic

In [ ]:
len(uncategorized)

We should be able to list all of these drug names in their pairs.

In [ ]:
repeated = set()
for drug in uncategorized:
    if drug not in repeated:
        print(drug, '||', list(drug_dict[drug])[0])
        repeated.add(drug)
        repeated.add(list(drug_dict[drug])[0])

With so few pairs, we can Google the names to determine which names of a pair are generic and brand names.

In [ ]:
# new_brands = [
#     'Clomid', 'Premarin', 'ParaGard', 'Natazia', 'NuvaRing', 'Femara', \
#     'Glucophage', 'Lysteda', 'Megace', 'Necon 1 / 50', 'ella'
# ]

# brands.extend(new_brands)

# len(brands)

In [ ]:
new_brands = [
    'ParaGard', 'Natazia', 'NuvaRing', 'Necon 1 / 50', 'ella'
]

brands.extend(new_brands)

len(brands)

In [ ]:
for drug in new_brands:
    generics.append(list(drug_dict[drug])[0])

len(generics)

At this point, we have sorted all the paired brand and generic drug names. What remains is to identify whether each of the single drug names is a generic or brand name.

Let's look at them.

In [ ]:
unpaired_drug_names

Simple Google search confirms these are both generic names, so we'll add them as such.

In [ ]:
generics.extend(unpaired_drug_names)

generics.sort()

Now we create a more universal drug naming system whereby every record is identified with its generic name.

In [ ]:
def generic_fix(drugList):
    if len(drugList) == 1 or drugList[0] in generics:
        return drugList[0]
    else:
        return drugList[1]

df['genericName'] = df.drugList.apply(lambda x: generic_fix(x))

In [ ]:
def full_brand_fix(drugList):
    if len(drugList) == 1:
        return None
    elif drugList[0] in brands:
        return drugList[0]
    else:
        return drugList[1]

df['fullBrandName'] = df.drugList.apply(lambda x: full_brand_fix(x))

In [ ]:
brand_dict = {}

for fullName in brands:
    name = fullName
    tail = name.split()[-1]
    while tail.isnumeric() or tail in ['Fe', 'Lo', 'One-Step', '/', '1.5', 'Contraceptive']:
        name = name[:len(name)-len(tail)-1]
        tail = name.split()[-1]
    head = name.split()[0]
    while head in ['Lo', '/']:
        name = name[len(head)+1:]
        head = name.split()[0]
    brand_dict[fullName] = name

df['shortBrandName'] = df.fullBrandName.apply(lambda x: None if x == None else brand_dict[x])

## Duplicates due to multiple user entry

Now we'll turn to more possible duplicate instances. We suspect the same user has copy-pasted an identical review multiple times when that verbatim review appears for the same condition and (generic) drug name with the same rating. Let's look at all such instances.

In [ ]:
len(df[df.duplicated(subset=['genericName', 'condition', 'review', 'rating'])])

In [ ]:
df[df.duplicated(subset=['genericName', 'condition', 'review', 'rating'])] \
[['genericName', 'condition', 'review', 'rating', 'date']]

The review texts all appear to be unique. As long as the review and its duplicate appear close in time to one another (within days), then these should be collapsed into a single review with the respective useful counts added together.

First we'll check on the dates. The following cell will show the respective dates of when these duplicated reviews appeared.

In [ ]:
for ind in df[df.duplicated(subset=['genericName', 'condition', 'review', 'rating'])].index:
    two_indices = list(df[df.review == df.loc[ind].review].index)
    print(df.loc[two_indices[0]].date, '... and ...', df.loc[two_indices[1]].date)

They're all identical dates except one that is a day apart.

We'll collapse these into single records and add the useful counts.

In [ ]:
for ind in df[df.duplicated(subset=['genericName', 'condition', 'review', 'rating'])].index:
    two_indices = list(df[df.review == df.loc[ind].review].index)
    x, y = two_indices[0], two_indices[1]
    count = int(df.loc[x].usefulCount + df.loc[y].usefulCount)
    df.at[x, 'usefulCount'] = count
    df.drop([y], inplace=True)

In [ ]:
# DO NOT re-run this cell out of sequence
# to use the dataframe as it was at this stage, un-comment, run, and re-comment the cell that follows after it.
df_bookmark_3 = df.copy()

In [ ]:
# df = df_bookmark_3.copy()

## Identifying birth control type

In [ ]:
birth_control_dict = {
    'IUD': ['Skyla', 'Mirena', 'Kyleena', 'Liletta', 'ParaGard'],
    'patch': ['Ortho Evra', 'Xulane'],
    'implantable': ['Implanon', 'Nexplanon'],
    'vaginal': ['NuvaRing'],
    'injectable': ['Depo-Provera', 'depo-subQ provera', 'Provera'],
    'emergency': ['Plan B', 'ella', 'Fallback Solo', 'Aftera', 'Take Action', 'Next Choice',
                       'My Way', 'EContra EZ'],
    'pill': ['Yasmin', 'Ortho Tri-Cyclen', 'Alesse', 'Aviane', 'Sprintec', 'Tri-Sprintec', 'Mircette',
             'Seasonique', 'Yaz', 'Lutera', 'Portia', 'Camila', 'Apri', 'Beyaz', 'Desogen', 'Kariva',
             'TriNessa', 'Zarah', 'Estarylla', 'Mononessa', 'Gianvi', 'Jolivette', 'Loestrin', 'Microgestin',
             'Ortho-Cyclen', 'Ortho-Novum', 'Necon', 'Femcon', 'Marlissa', 'Aubra', 'Viorele', 'Vestura',
             'Norlyda', 'Ortho Cyclen', 'Lybrel', 'Pirmella', 'Larin', 'Tarina', 'Previfem', 'Tri-Estarylla',
             'Lessina', 'Elinest', 'Cryselle', 'Ortho-Cept', 'Falmina', 'Altavera', 'Tri-Lo-Marzia', 'Taytulla',
             'CamreseLo', 'Philith', 'Dasetta', 'Gildess', 'Ovral', 'Jencycla', 'Tri-Linyah', 'Enskyce',
             'Orsythia', 'Sronyx', 'Velivet', 'Reclipsen', 'Nikki', 'Levlen', 'Loryna', 'Juleber', 'Trivora',
             'Zenchent', 'Tri-Previfem', 'Lyza', 'Seasonale', 'Mono-Linyah', 'Alyacen', 'Opcicon']
}

In [ ]:
def method_fix_1(shortBrandName):
    for method in birth_control_dict:
        if shortBrandName in birth_control_dict[method]:
            return method
    else:
        return None

df['method'] = df.shortBrandName.apply(lambda x: None if x == None else method_fix_1(x))

In [ ]:
generic_dict = {
    'patch': ['Ethinyl estradiol / norelgestromin'],
    'IUD': ['Copper'],
    'implantable': ['Etonogestrel'],
    'emergency': ['Ulipristal'],
    'vaginal': ['Ethinyl estradiol / etonogestrel'],
    'pill': ['Ethinyl estradiol / levonorgestrel',
             'Drospirenone / ethinyl estradiol / levomefolate calcium',  'Mestranol / norethindrone',
             'Ethinyl estradiol / norgestimate', 'Ethinyl estradiol / norethindrone',
             'Norethindrone', 'Drospirenone / ethinyl estradiol', 'Desogestrel / ethinyl estradiol',
             'Ethinyl estradiol / norgestrel', 'Ethinyl estradiol / folic acid / levonorgestrel',
             'Ethinyl estradiol / ethynodiol', 'Dienogest / estradiol'],
    'injectable': ['Medroxyprogesterone']
}

In [ ]:
def method_fix_2(generic):
    for method in generic_dict:
        if generic in generic_dict[method]:
            return method
    else:
        return None

df['method'] = df.apply(lambda x: method_fix_2(x.genericName) if x.method == None else x.method, axis=1)

In [ ]:
df[
    (df.method.isna())
].genericName.value_counts()

In [ ]:
for ind in df[df.genericName == 'Nonoxynol 9'].index:
    show_review(ind)

We could create a spermicide label, but there would only be two records for it, so we'll just drop them.

In [ ]:
df.drop([172606, 209857], inplace=True)

In [ ]:
for ind in df[
    (df.genericName == 'Levonorgestrel') &
    (df.method.isna())
].index[:10]:
    show_review(ind)

All of these examples describe emergency contraception.

In [ ]:
generic_dict['emergency'].append('Levonorgestrel')
df['method'] = df.apply(lambda x: method_fix_2(x.genericName) if x.method == None else x.method, axis=1)

In [ ]:
df[
    (df.method.isna())
].genericName.value_counts()

In [ ]:
# DO NOT re-run this cell out of sequence
# to use the dataframe as it was at this stage, un-comment, run, and re-comment the cell that follows after it.
df_bookmark_4 = df.copy()

In [ ]:
# df = df_bookmark_4.copy()

# Exploration

Relevant columns now are:
- condition (emergency v birth control, although this is flawed)
- rating
- date
- usefulCount
- shortBrandName
- method

## Data over time

In [ ]:
# Calculate average rating for each method
average_ratings = df.groupby('method')['rating'].mean().reset_index()

# Create a bar chart using Seaborn
plt.figure(figsize=(14, 6))

# Set the font scale to increase text size
sns.set(font_scale=1.5)
sns.barplot(
    x='method',
    y='rating',
    data=average_ratings,
    order=average_ratings.sort_values('rating', ascending=False)['method'],
    palette='deep'
)
plt.xlabel('Method')
plt.ylabel('Average Rating')
plt.title('Average Rating for Each Method')
plt.show()


In [ ]:
# Extract year from datetime
df['year'] = df['datetime'].dt.year

# Group by 'year' and 'method', calculate the average rating for each group
average_ratings = df.groupby(['year', 'method'])['rating'].mean().reset_index()

# Pivot the DataFrame to have 'method' as columns and 'year' as index
pivot_table = average_ratings.pivot_table(index='year', columns='method', values='rating')

# Plot the data
pivot_table.plot(kind='line', marker='o', figsize=(14, 6))
plt.title('Average Rating of Each Method by Year')
plt.xlabel('Year')
plt.ylabel('Average Rating')
plt.legend(title='Method', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

Observations:
- Ratings generally declined after the number of ratings increased significantly in 2015.
- Emergency contraception has largely been more popular than other methods.
- IUDs, patches, and vaginal contraceptives tend to be more popular than pills and implantables.
- Injectable methods are consistently the least popular.

## Rating and upvotes

In [ ]:
# # Calculate total counts for each rating
# rating_counts = df['rating'].value_counts().reset_index().rename(columns={'index': 'rating', 'rating': 'count'})

# # Create a bar chart using Seaborn
# plt.figure(figsize=(10, 6))
# sns.barplot(x='rating', y='count', data=rating_counts, palette='muted')
# plt.xlabel('Rating')
# plt.ylabel('Total Counts')
# plt.title('Total Counts for Each Rating')
# plt.show()

# Count the occurrences of each rating
rating_counts = df['rating'].value_counts()

# Sort the index for better visualization
rating_counts = rating_counts.sort_index()

# Plotting the bar graph
plt.figure(figsize=(14, 6))
plt.bar(rating_counts.index, rating_counts.values, color='skyblue')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.title('Distribution of Ratings')
plt.show()

Observations:
- There tend to be more positive ratings.
- Negative ratings are concentrated on the worst rating (1.0).
- There is a slight peak at 5.0. This indicates an attempt at neutrality.

In light of this distribution, we could recommend a 3-, 4-, or 5-point rating system that might be a better fit.

In [ ]:
# Calculate average usefulCount for each rating
average_useful_count = df.groupby('rating')['usefulCount'].mean().reset_index()

# Create a bar chart using Seaborn
plt.figure(figsize=(14, 6))
sns.barplot(x='rating', y='usefulCount', data=average_useful_count, palette='viridis')
plt.xlabel('Rating')
plt.ylabel('Average Useful Count')
plt.title('Average Useful Count for Each Rating')
plt.show()

Observations:
- There is no real gain in usefulness other than for ratings 8 and above, which we saw earlier are also more prevalent than all other ratings except for 1.0, the lowest.

In [ ]:
# Calculate average usefulCount for each rating
average_useful_count = df[df.usefulCount > df.usefulCount.quantile(.80)].groupby('rating')['usefulCount'].mean().reset_index()

# Create a bar chart using Seaborn
plt.figure(figsize=(14, 6))
sns.barplot(x='rating', y='usefulCount', data=average_useful_count, palette='viridis')
plt.xlabel('Rating')
plt.ylabel('Average Useful Count')
plt.title('Average Useful Count for Each Rating — Top 20% of Useful Counts')
plt.show()

There's a relative spike among ratings of 3.0. Let's look at some highly upvoted reviews with a rating of 3.0.

In [ ]:
for ind in df[df.rating == 3].sort_values(by='usefulCount', ascending=False).index[:10]:
    show_review(ind)

Nothing jumps out about these reviews except that they are certainly negative and could all probably have been 1.0 ratings.

## Term frequency analysis

In [ ]:
# obtain the standard list of stopwords
nltk.download('stopwords', quiet=True)
# start our own list of stopwords with these words
stop_list = stopwords.words('english')

# consider trimming the stop_list to keep the following for the purposes of certain n-grams:
# 44-59 be/have/do verbs
# 64-178 prepositions/subordinate conjunctions/modals

# add punctuation characters
for char in string.punctuation:
    stop_list.append(char)
# add empty string
stop_list.extend(['', 'ha', 'wa', '``', "''"])
# PUT THIS SOMEWHERE ELSE
lemmatizer = WordNetLemmatizer()

In [ ]:
reviews, corpus, tokens = {}, {}, {}

reviews['vaginal'] = df[df.method == 'vaginal'].review_lower.tolist() # a list of reviews

corpus['vaginal'] = ' '.join(reviews['vaginal'])

tokens['vaginal'] = nltk.word_tokenize(corpus['vaginal'])



In [ ]:
reviews, corpus, tokens, tokens_joined = {}, {}, {}, {}

for method_ in df.method.unique():
    reviews[method_] = df[df.method == method_].review_lower.tolist() # a list of all reviews of one method
    corpus[method_] = ' '.join(reviews[method_]) # the above as a single string with spaces
    tokens[method_] = nltk.word_tokenize(corpus[method_]) # the above as a list of words
    tokens_joined[method_] = ' '.join(tokens[method_]) # the above as a single string with spaces

tokens_joined_list = [tokens_joined[method_] for method_ in df.method.unique()]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_list)
tfidf_matrix = tfidf_vectorizer.fit_transform(tokens_joined_list)

for i, doc in enumerate(documents):
    print(f"Important words in Document {i+1}:")
    feature_names = tfidf_vectorizer.get_feature_names_out()
    feature_index = tfidf_matrix[i, :].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
    top_words = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:25]  # Adjust 5 to desired number of top words
    for word_index, score in top_words:
        print(f"{feature_names[word_index]} (TF-IDF Score: {score:.2f})")
    print()


In [ ]:
# a function that takes a list of documents and 1) tokenizes them, 2) lemmatizes them, and 3) removes stopwords

# for example, to execute this function use make_tokens(df.review.tolist())

# IN: a list of documents
# OUT: a list of tokens

def make_tokens(docs_list, stop_list=stop_list):
    # join documents into a single string
    docs_joined = ' '.join(docs_list)
    # tokenize the single string into a list of tokens
    tokens = nltk.word_tokenize(docs_joined)
    # lemmatize the list of tokens
    tokens_lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    # stop the list of tokens
    tokens_stopped = [word for word in tokens_lemmatized if \
                      word not in stop_list]
    return tokens_stopped

In [ ]:
# a function that generates a word cloud of a given list of words
def make_wordcloud(wordlist, colormap='Greens', title=None):
    # instantiate wordcloud
    wordcloud = WordCloud(
        width=600,
        height=400,
        colormap=colormap,
        collocations = True
    )
    return wordcloud.generate(','.join(wordlist))

In [ ]:
# a function that plots the word cloud
def plot_wordcloud(wordcloud):
    # plot wordcloud
    plt.figure(figsize = (12, 15)) 
    plt.imshow(wordcloud) 
    plt.axis('off');

In [ ]:
df['review_lower'] = df['review'].str.lower()

In [ ]:
def method_summary(method_):
    tokens = make_tokens(df[df.method == method_].review_lower.tolist())
    plot_wordcloud(make_wordcloud(tokens))
    print(FreqDist(tokens).most_common(25))

In [ ]:
method_summary('injectable')

In [ ]:
method_tokens = []

for method_ in df.method.unique():
    reviews = df[df.method == method_].review_lower.tolist() # a list of documents
    method_tokens.append(make_tokens(df[df.method == method_].review_lower.tolist()))

In [ ]:
# def get_tfidf_top_features(documents,n_top=10):
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,  stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(method_tokens)
#     importance = np.argsort(np.asarray(tfidf.sum(axis=0)).ravel())[::-1]
#     tfidf_feature_names = np.array(tfidf_vectorizer.get_feature_names())
#     return tfidf_feature_names[importance[:n_top]]

In [ ]:
get_tfidf_top_features(method_tokens)

In [ ]:
len(method_tokens)

What is good or bad about each method

What makes good (or bad) reviews useful about each method

For each birth control method, make a function that summarizes what people are saying good and bad about it.

Create a weighted rating (rating times usefulCount) and use this to inform topic modeling.

explore each feature vs. rating and each feature vs. useful

explore topic modeling

In [ ]:
for ind in df[df.method == 'implantable'].sort_values(by='usefulCount', ascending=False)[:10].index:
    show_review(ind)

In [ ]:
df.method.value_counts()

In [ ]:
pill_tokens = make_tokens(df[df.method == 'pill'].review_lower.tolist(), stop_list=stop_list)

In [ ]:
implantable_tokens = make_tokens(df[df.method == 'implantable'].review_lower.tolist(), stop_list=stop_list)

In [ ]:
plot_wordcloud(make_wordcloud(pill_tokens))

In [ ]:
plot_wordcloud(make_wordcloud(implantable_tokens))

In [ ]:
df.groupby('method').genericName.nunique()

In [ ]:
df.groupby('method').shortBrandName.nunique()

In [ ]:
df.groupby('method').rating.mean()

In [ ]:
df.groupby('method').usefulCount.mean()

This shows the average useful count by rating from 1 to 10. It seems that ratings of 8 and above are considered more useful than the others.

In [ ]:
threshold = 20

uv = list(df.usefulCount.values)
uv.sort(reverse=True)
n = 0
while sum(uv[:n]) < threshold / 100 * df.usefulCount.sum():
    n += 1
print(str(int(n * 100 / len(df)))+'%','of the reviews received', str(threshold)+'%', 'of all the upvotes.')
print('This includes reviews with', uv[n], 'or more upvotes')

In [ ]:
df[df.usefulCount >= 41].rating.value_counts(normalize=True)

In [ ]:
threshold = 40

uv = list(df.usefulCount.values)
uv.sort(reverse=True)
n = 0
while sum(uv[:n]) < threshold / 100 * df.usefulCount.sum():
    n += 1
print(str(int(n * 100 / len(df)))+'%','of the reviews received', str(threshold)+'%', 'of all the upvotes.')
print('This includes reviews with', uv[n], 'or more upvotes')

In [ ]:
df[df.usefulCount >= 20].rating.value_counts(normalize=True)

In [ ]:
threshold = 60

uv = list(df.usefulCount.values)
uv.sort(reverse=True)
n = 0
while sum(uv[:n]) < threshold / 100 * df.usefulCount.sum():
    n += 1
print(str(int(n * 100 / len(df)))+'%','of the reviews received', str(threshold)+'%', 'of all the upvotes.')
print('This includes reviews with', uv[n], 'or more upvotes')

In [ ]:
df[df.usefulCount >= 11].rating.value_counts(normalize=True)

In [ ]:
threshold = 80

uv = list(df.usefulCount.values)
uv.sort(reverse=True)
n = 0
while sum(uv[:n]) < threshold / 100 * df.usefulCount.sum():
    n += 1
print(str(int(n * 100 / len(df)))+'%','of the reviews received', str(threshold)+'%', 'of all the upvotes.')
print('This includes reviews with', uv[n], 'or more upvotes')

In [ ]:
df[df.usefulCount >= 6].rating.value_counts(normalize=True)

In [ ]:
threshold = 95

uv = list(df.usefulCount.values)
uv.sort(reverse=True)
n = 0
while sum(uv[:n]) < threshold / 100 * df.usefulCount.sum():
    n += 1
print(str(int(n * 100 / len(df)))+'%','of the reviews received', str(threshold)+'%', 'of all the upvotes.')
print('This includes reviews with', uv[n], 'or more upvotes')

In [ ]:
df[df.usefulCount >= 3].rating.value_counts(normalize=True)

In [ ]:
X = range(0,len(df),100)
Y = []
for x in X:
    Y.append(sum(uv[:x]))
fig, ax = plt.subplots()
ax.plot(X,Y);

This shows the cumulative sum of upvotes. For example, if we go 5% of the reviews in from the left on the x-axis, we would reach up to 33% of the total upvotes on the y-axis.

# feature engineering ideas

- word count
- character count
- words in all caps
- average word length
- whether words are in English (spelled correctly)
- whether it includes characters such as exclamation points, question marks, (especially repeatedly), and emoticons
- whether it mentions the brand or generic name in the review
- whether it uses slang versus technical language
- whether it mentions touchy subjects such as abortion
- ngrams
- does it mention switching treatments, comparison to other methods

In [ ]:
engineered_features = []

In [ ]:
df['raw_tokens'] = df.review.apply(lambda x: nltk.word_tokenize(x))

In [ ]:
df['review_lower'] = df['review'].str.lower()

In [ ]:
def plot_useful(feature, low=0.15, high=0.85):
    X = range(int(df[feature].quantile(low)), int(df[feature].quantile(high)))
    Y = []
    for x in X:
        Y.append(df[df[feature] == x].usefulCount.mean())
    fig, ax = plt.subplots()
    ax.plot(X,Y);

# all caps words

In [ ]:
def all_caps_fix(tokens_list):
    count = 0
    for token in tokens_list:
        if len(token) > 2 and token.isupper():
            count += 1
    return count

df['all_caps'] = df.raw_tokens.apply(lambda x: all_caps_fix(x))

In [ ]:
df.all_caps.value_counts()[:10]

In [ ]:
plot_useful('all_caps', high=0.99)

In [ ]:
engineered_features.append('all_caps')

# no caps usage

In [ ]:
df['no_caps'] = df.apply(lambda x: 1 if x.review == x.review_lower else 0, axis=1)

In [ ]:
df.no_caps.value_counts()

In [ ]:
df[df.no_caps == 1].usefulCount.mean()

In [ ]:
df[df.no_caps == 0].usefulCount.mean()

In [ ]:
engineered_features.append('no_caps')

# exclaim count

In [ ]:
df['exclaim_count'] = df.raw_tokens.apply(lambda x: x.count('!'))

In [ ]:
df.exclaim_count.value_counts()[:10]

In [ ]:
plot_useful('exclaim_count', high=0.995)

In [ ]:
engineered_features.append('exclaim_count')

# question count

In [ ]:
df['question_count'] = df.raw_tokens.apply(lambda x: x.count('?'))

In [ ]:
df.question_count.value_counts()

In [ ]:
plot_useful('question_count', low=0.01, high=0.999)

In [ ]:
engineered_features.append('question_count')

# word count

In [ ]:
df['word_count'] = df.raw_tokens.apply(lambda x: len(x))

In [ ]:
plot_useful('word_count')

In [ ]:
engineered_features.append('word_count')

# character count

In [ ]:
df['character_count'] = df.review.apply(lambda x: len(x))

In [ ]:
plot_useful('character_count')

In [ ]:
engineered_features.append('character_count')

# characters per word

In [ ]:
df['cpw'] = 100 * df.character_count / df.word_count

In [ ]:
df.cpw.value_counts()

In [ ]:
plot_useful('cpw', low=0.01, high=0.99)

In [ ]:
engineered_features.append('cpw')

# age disclosure

In [ ]:
def age_disclosure_feature(review):
    if re.search('[1-4][0-9] +y(ea)?rs? +(old|of age)', review) or \
    re.search('(pregnant( +at)?|age( +of)?|being) +[1-4][0-9]', review) or \
    re.search('(teen|ty|one|two|three|four|five|six|seven|eight|nine) +y(ea)?rs? +(old|of +age)', review) or \
    re.search('i +(am +|was +|will +be +|would have been +)(not even +)?(now +)?(only +)?(just +)?(maybe +)?[1-4][0-9](\s|\.|\!|\?)', review):
        return True
    else:
        return False

df['age_disclosure'] = df.review_lower.apply(lambda x: 1 if age_disclosure_feature(x) else 0)

In [ ]:
df.age_disclosure.value_counts()

In [ ]:
df[df.age_disclosure == 1].usefulCount.mean()

In [ ]:
df[df.age_disclosure == 0].usefulCount.mean()

In [ ]:
engineered_features.append('age_disclosure')

In [ ]:
# DO NOT re-run this cell out of sequence
# to use the dataframe as it was at this stage, un-comment, run, and re-comment the cell that follows after it.
df_bookmark_5 = df.copy()

In [ ]:
# df = df_bookmark_5.copy()

# functions

# separate pos / neg

In [ ]:
df_neg = df[df.rating < 4]

In [ ]:
df_neg.shape

In [ ]:
neg_reviews = df_neg.review.tolist()

In [ ]:
lemmatizer = WordNetLemmatizer()

neg_tokens = make_tokens(neg_reviews)

In [ ]:
ff = ["me", 'you', 'th"ey', 'we', "we"]

In [ ]:
barf = ['"', "I've", 'tried', 'a', 'few']

In [ ]:
def make_wordcloud(wordlist, colormap='Greens', title=None):
    # instantiate wordcloud
    wordcloud = WordCloud(
        width=600,
        height=400,
        colormap=colormap,
        collocations = True
    )
    return wordcloud.generate(','.join(wordlist))

def plot_wordcloud(wordcloud):
    # plot wordcloud
    plt.figure(figsize = (12, 15)) 
    plt.imshow(wordcloud) 
    plt.axis('off');

In [ ]:
plot_wordcloud(make_wordcloud(ff))

In [ ]:
uv = list(df_neg.usefulCount.values)
uv.sort(reverse=True)

In [ ]:
X = range(0,len(df_neg),50)
Y = []
for x in X:
    Y.append(sum(uv[:x]))
fig, ax = plt.subplots()
ax.plot(X,Y);

## Results function

We'll create a function that takes the pipeline we've created and displays only the results we're interested in.

In [ ]:
def model_results(pipeline):
    # fit the pipeline to the training data
    pipeline.fit(X_train, y_train)
    # generate predictions for the test data
    y_pred = pipeline.predict(X_test)
    # display the training and test accuracy scores
    print(f"Training Score: {round(pipeline.score(X_train, y_train),4)} \
    \nTest Score:     {round(pipeline.score(X_test, y_test),4)}")
    
    # plot the normalized confusion matrix
    plot_confusion_matrix(estimator=pipeline, X=X_test, y_true=y_test, cmap='Greens', 
                          normalize='true', 
                          display_labels=['Non-positive', 'Positive'])

# save and reload preprocessed set

At this stage we will save and reload the preprocessed set in order to avoid taking the time to repeat earlier work everytime we open the notebook.

The saved version has restored or deleted all records with missing condition labels.

We have established pairs in the list `twins` but we have NOT yet deleted either member of any pair or dealt with the confusion between brand and generic drug names.

The size of the dateframe is nearly the same as its original version, roughly 215,000 records.

In the future, store the functions with this?

Store the engineered features list.

In [ ]:
filepath = Path('../data/preprocessed.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath)

In [ ]:
%store engineered_features

In [ ]:
df = pd.read_csv('../data/preprocessed.csv')
df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
%store -r engineered_features

# Modeling

I'll carefully put the pieces together one cell at a time. If it all works, I can combine them later.

* define target feature using thresholds

In [ ]:
useful_threshold = [1, 12]

print(len(df[df.usefulCount <= useful_threshold[0]]), len(df[df.usefulCount >= useful_threshold[1]]))

In [ ]:
df.drop(df[
    (df.usefulCount > useful_threshold[0]) & \
    (df.usefulCount < useful_threshold[1])
].index, inplace=True)

df['target'] = df.usefulCount.apply(lambda x: 1 if x >= useful_threshold[1] else 0)

In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(df[['review'] + engineered_features], df['target'], test_size=0.2, random_state=SEED)

In [ ]:
# save this value to compare to future model crossval scores
plurality_cv = round(y_train.value_counts(normalize=True)[1],4)
# show the sentiment breakdown
round(y_train.value_counts(normalize=True),4)

----------------------------------

In [ ]:
# reset variables

text_preprocessor = None
numerical_preprocessor = None
preprocessor = None
pipeline = None
accuracy = None
feature_names = None
coefficients = None
decision_function_values = None
importance_df = None
feature_importance = None

In [ ]:
# obtain the standard list of stopwords
nltk.download('stopwords', quiet=True)
# start our own list of stopwords with these words
stop_list = stopwords.words('english')
# add punctuation characters
# for char in string.punctuation:
#     stop_list.append(char)
# add empty string
# stop_list.extend([''])

In [ ]:
# stop words to keep
# 44-59 be/have/do verbs
# 64-178 prepositions/subordinate conjunctions/modals
stop_list = stop_list[:44] + stop_list[60:64]

In [ ]:
# variables of interest

max_features = None
stop_words = stop_list
ngram_range = (2,4)
model = LogisticRegression(max_iter=1000)
# model = SVC(kernel='linear')
# model = MultinomialNB() # does not work
# model = GaussianNB() # does not work
# model = DecisionTreeClassifier(random_state=SEED)
# model = BaggingClassifier()
# model = RandomForestClassifier(random_state=SEED)
# model = AdaBoostClassifier(random_state=SEED)
# model = GradientBoostingClassifier(random_state=SEED)
# model = XGBClassifier(random_state=SEED)
n_features = 25
title = '_____ Model'

In [ ]:
text_preprocessor = TfidfVectorizer(
    max_features=max_features,
    stop_words=stop_words,
    ngram_range=ngram_range
)

In [ ]:
numerical_preprocessor = StandardScaler()

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_preprocessor, 'review'),
        ('numerical', numerical_preprocessor, engineered_features)
    ]
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

In [ ]:
%%time
pipeline.fit(X_train, y_train);

In [ ]:
accuracy = pipeline.score(X_test, y_test)

In [ ]:
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
feature_names = preprocessor.named_transformers_['text'].get_feature_names_out().tolist() + engineered_features

In [ ]:
# the success of this code seems to depend on whether we feed it a dense or sparse matrix
# supposedly the .ravel() method will handle either

# this works for SVC, LogisticRegression

# --------------------------------------------------------
if hasattr(model, 'feature_importances_'):
    coefficients = model.feature_importances_
elif hasattr(model.coef_, 'toarray'):
    coefficients = model.coef_.toarray().flatten()
else:
    coefficients = model.coef_.flatten()
# --------------------------------------------------------

# this works for DecisionTreeClassifier, RandomForestClassifier, AdaBoostClassifier
# seems meaningless in the case of AdaBoost

# --------------------------------------------------------
# coefficients = model.feature_importances_
# --------------------------------------------------------

# this works for BaggingClassifier
# (but it might not mean anything?)

# --------------------------------------------------------
# coefficients = np.mean([
#     tree.feature_importances_ for tree in model.estimators_
# ], axis=0)
# --------------------------------------------------------

# coefficients = model.coef_.toarray().flatten()  # Assuming sparse matrix

# coefficients = model.coef_.flatten()  # Assuming dense matrix

# coefficients = model.coef_.ravel()  # should work for dense or sparse matrix

# try:
#     # Try to access coef_ (only available for linear kernels)
#     coefficients = model.coef_.flatten()
# except AttributeError:
#     # Handle the case for non-linear kernels
#     decision_function_values = pipeline.decision_function(X_train)
#     coefficients = (preprocessor.transform(X_train).T @ decision_function_values * y_train).sum(axis=1)

In [ ]:
importance_df = pd.DataFrame(feature_names, columns=['Word'])
importance_df['Importance'] = np.e**(abs(coefficients))
importance_df['Coefficient'] = coefficients

In [ ]:
feature_importance = importance_df.sort_values(
    by = ["Importance"], ascending=False
).head(n_features)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10), ncols=2)
ax[0].set_title(f'Coefficients for {title}')
ax[0].set_ylabel('Word')
ax[0].set_xlabel('Coefficient')
sns.barplot(x='Coefficient', y='Word', data=feature_importance, 
            palette='coolwarm', ax=ax[0])
#plotting feature importances
ax[1].set_title(f'Feature Importances for {title}')
ax[1].set_ylabel('Word')
ax[1].set_xlabel('Importance')
sns.barplot(x='Importance', y='Word', data=feature_importance, 
            palette='coolwarm', ax=ax[1])
plt.tight_layout();

# Evaluation

# Recommendations

Don't know where else to put this right now. At least one review, 27228, is glowingly positive but gave a 1.0 rating, clearly misunderstanding the rating system.

## Use a different rating system

No more than 5 points.

# Further Inquiry